**ODBC Connection to Azure SQL DB with Azure AD User Access Token**

Install msodbcsql17 with pyodbc

In [0]:
%sh
curl https://packages.microsoft.com/keys/microsoft.asc | apt-key add -
curl https://packages.microsoft.com/config/ubuntu/16.04/prod.list > /etc/apt/sources.list.d/mssql-release.list 
apt-get update
ACCEPT_EULA=Y apt-get install msodbcsql17
apt-get -y install unixodbc-dev
sudo apt-get install python3-pip -y
pip3 install --upgrade pyodbc

% Total % Received % Xferd Average Speed Time Time Time Current
 Dload Upload Total Spent Left Speed

 0 0 0 0 0 0 0 0 --:--:-- --:--:-- --:--:-- 0
100 983 100 983 0 0 33896 0 --:--:-- --:--:-- --:--:-- 33896
Warning: apt-key output should not be parsed (stdout is not a terminal)
OK
 % Total % Received % Xferd Average Speed Time Time Time Current
 Dload Upload Total Spent Left Speed

 0 0 0 0 0 0 0 0 --:--:-- --:--:-- --:--:-- 0
 0 0 0 0 0 0 0 0 --:--:-- --:--:-- --:--:-- 0
100 79 100 79 0 0 3291 0 --:--:-- --:--:-- --:--:-- 3160
Get:1 https://packages.microsoft.com/ubuntu/16.04/prod xenial InRelease [4,003 B]
Ign:2 http://repos.azulsystems.com/ubuntu stable InRelease
Hit:3 http://repos.azulsystems.com/ubuntu stable Release
Get:4 https://packages.microsoft.com/ubuntu/16.04/prod xenial/main amd64 Packages [194 kB]
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:8 https://repos.azul.com/zulu/deb stable InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:10 http://security.ubuntu.com/ubuntu bionic-security/multiverse amd64 Packages [15.3 kB]
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:12 http://security.ubuntu.com/ubuntu bionic-security/restricted amd64 Packages [237 kB]
Get:13 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 Packages [2,136 kB]
Get:14 http://security.ubuntu.com/ubuntu bionic-security/universe amd64 Packages [1,372 kB]
Get:15 http://security.ubuntu.com/ubuntu bionic-security/main amd64 Packages [1,816 kB]
Get:16 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 Packages [2,244 kB]
Get:17 http://archive.ubuntu.com/ubuntu bionic-updates/restricted amd64 Packages [266 kB]
Get:18 http://archive.ubuntu.com/ubuntu bionic-updates/multiverse amd64 Packages [53.8 kB]
Fetched 8,590 kB in 3s (3,197 kB/s)
Reading package lists...
Reading package lists...
Building dependency tree...
Reading state information...
The following packages were automatically installed and are no longer required:
 libcap2-bin libpam-cap zulu-repo
Use 'sudo apt autoremove' to remove them.
The following NEW packages will be installed:
 msodbcsql17
0 upgraded, 1 newly installed, 0 to remove and 52 not upgraded.
Need to get 743 kB of archives.
After this operation, 0 B of additional disk space will be used.
Get:1 https://packages.microsoft.com/ubuntu/16.04/prod xenial/main amd64 msodbcsql17 amd64 17.6.1.1-1 [743 kB]
debconf: delaying package configuration, since apt-utils is not installed
Fetched 743 kB in 0s (14.5 MB/s)
Selecting previously unselected package msodbcsql17.
(Reading database ... 
(Reading database ... 5%
(Reading database ... 10%
(Reading database ... 15%
(Reading database ... 20%
(Reading database ... 25%
(Reading database ... 30%
(Reading database ... 35%
(Reading database ... 40%
(Reading database ... 45%
(Reading database ... 50%
(Reading database ... 55%
(Reading database ... 60%
(Reading database ... 65%
(Reading database ... 70%
(Reading database ... 75%
(Reading database ... 80%
(Reading database ... 85%
(Reading database ... 90%
(Reading database ... 95%
(Reading database ... 100%
(Reading database ... 91333 files and directories currently installed.)
Preparing to unpack .../msodbcsql17_17.6.1.1-1_amd64.deb ...
debconf: unable to initialize frontend: Dialog
debconf: (Dialog frontend will not work on a dumb terminal, an emacs shell buffer, or without a controlling terminal.)
debconf: falling back to frontend: Readline
Unpacking msodbcsql17 (17.6.1.1-1) ...
Setting up msodbcsql17 (17.6.1.1-1) ...
Reading package lists...
Building dependency tree...
Reading state information...
The following packages were automatically installed and are no longer required:
 libcap2-bin libpam-cap zulu-repo
Use 'sudo apt autoremove' to remove them.
The following additional packages will be installed:
 libodbc1 odbcinst odbcinst1debian2 unixodbc
Suggested packages:
 uni

Import libraries

In [0]:
# AADTokenCredentials for multi-factor authentication
from msrestazure.azure_active_directory import AADTokenCredentials

In [0]:
# Import other necessary libraries
import adal, uuid, time
import pandas as pd
import pyodbc
import struct

Declar variables

In [0]:
# Define the Azure AD Tenant ID
tenant = "PLEASE_ENTER_YOUR_OWNED_AZURE_AD_TENANT_ID"
# Define the Azure SQL DB Server name.
sql_hostname = "PLEASE_ENTER_YOUR_OWNED_AZURE_SQL_DB_SERVER_NAME"
# Define the Azure SQL DB Database name.
database_name = "PLEASE_ENTER_YOUR_OWNED_AZURE_SQL_DB_NAME"
# Misc
authority_host_uri = "https://login.microsoftonline.com"
authority_uri = authority_host_uri + '/' + tenant
# The client id used above is a well known that already exists for all azure services. While it makes the sample code easy to use,
# for production code you should use generate your own client ids for your application.
client_id = "04b07795-8ddb-461a-bbee-02f9e1bf7b46"
resource_app_id_url = "https://database.windows.net/"

Initiate Azure AD Authenication with Device Code

In [0]:
# Perfom Azure AD authenication to obtain access token for Azure SQL DB access.
context = adal.AuthenticationContext(authority_uri, api_version=None)
code = context.acquire_user_code(resource_app_id_url, client_id)
print(code['message'])
token = context.acquire_token_with_device_code(resource_app_id_url, code, client_id)

To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code HU43RJ2E7 to authenticate.

Declar ODBC connection string

In [0]:
# Construct ODBC connection string
odbcConnstr = "DRIVER={};SERVER={};DATABASE={}".format("ODBC Driver 17 for SQL Server", sql_hostname, database_name)

Make ODBC Connection to Azure SQL Database with Azure AD access token

In [0]:
# Make ODBC connection with Azure AD access token
tokenx = bytes(token["accessToken"], "UTF-8")
exptoken = b"";
for i in tokenx:
 exptoken += bytes({i});
 exptoken += bytes(1);
tokenstruct = struct.pack("=i", len(exptoken)) + exptoken;
conn = pyodbc.connect(odbcConnstr, attrs_before = { 1256:tokenstruct });
# conn = pyodbc.connect(jdbcConnstr, attrs_before = { 1256:bytearray(tokenstruct) });
# Make ODBC query to Azure SQL Database
query = "SELECT * FROM diabetes"
diabetes_df_sql = pd.read_sql(query, conn)
diabetes_df_sql.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


Convert Panda dataframe into Spark dataframe

In [0]:
sparkDf = spark.createDataFrame(diabetes_df_sql);

Display Spark dataframe

In [0]:
display(sparkDf)

Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
6,148,72,35,0,33.6,0.627,50,1
1,85,66,29,0,26.6,0.351,31,0
8,183,64,0,0,23.3,0.672,32,1
1,89,66,23,94,28.1,0.167,21,0
0,137,40,35,168,43.1,2.288,33,1
5,116,74,0,0,25.6,0.201,30,0
3,78,50,32,88,31,0.248,26,1
10,115,0,0,0,35.3,0.134,29,0
2,197,70,45,543,30.5,0.158,53,1
8,125,96,0,0,0,0.232,54,1
